In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
       os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import os
from PIL import Image
import openslide

In [ ]:
train_df = pd.read_csv("/kaggle/input/prostate-cancer-grade-assessment/train.csv").set_index('image_id')

In [ ]:
train_df.head(10)

### Image Visualization
--------------------------

In [ ]:
train_img_path = "/kaggle/input/prostate-cancer-grade-assessment/train_images"
label_path = "/kaggle/input/prostate-cancer-grade-assessment/train_label_masks"

train_img = [img for img in os.listdir(train_img_path)]
train_label = [label for label in os.listdir(label_path)]

train_img = list(sorted(train_img))
train_label = list(sorted(train_label))

SIZE = 300

def preprocessing_img(img):
    slide = openslide.OpenSlide(img)
    img = np.array(slide.get_thumbnail(size=(SIZE, SIZE)))
    img = Image.fromarray(img)
    img = img.resize((SIZE, SIZE))
    img = np.array(img)
    return img

img_array = []
for i in range(10):
    img = train_img_path + "/" + train_img[i]
    img = preprocessing_img(img)
    img_array.append(img)

In [ ]:
plt.imshow(img_array[0])

In [ ]:
img_test = img_array[0]
img_test.shape

In [ ]:
train_img[0]

In [ ]:
isup = train_df.iloc[0]['isup_grade']

## Pre Processing

### Denoising image

In [ ]:
import cv2

denoise = cv2.fastNlMeansDenoisingColored(img_test,None,10,10,7,21)


In [ ]:
f = plt.figure(figsize=(15,15))
f.add_subplot(2, 1, 1).set_title(f'Original Image isup = {isup}')
plt.imshow(img_array[0], cmap = "gray")
f.add_subplot(2, 1, 2).set_title(f'Filtered Image isup = {isup}')
plt.imshow(denoise, cmap = "gray")
plt.show()

#### Not too sure if image denoising will be helpful so will not use it for now 

- Thinking that specific values of color in image may factor into calculation

## Image Processing

In [ ]:
def RGB2HEX(color):
    return "#{:02x}{:02x}{:02x}".format(int(color[0]), int(color[1]), int(color[2]))

In [ ]:
def get_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

In [ ]:
modified_image = cv2.resize(img_array[0], (600, 400), interpolation = cv2.INTER_AREA)
modified_image = modified_image.reshape(modified_image.shape[0]*modified_image.shape[1], 3)

In [ ]:
from sklearn.cluster import KMeans
from collections import Counter
from skimage.color import rgb2lab, deltaE_cie76


clf = KMeans(n_clusters =8)
labels = clf.fit_predict(modified_image)

In [ ]:
counts = Counter(labels)

center_colors = clf.cluster_centers_
# We get ordered colors by iterating through the keys
ordered_colors = [center_colors[i] for i in counts.keys()]
hex_colors = [RGB2HEX(ordered_colors[i]) for i in counts.keys()]
rgb_colors = [ordered_colors[i] for i in counts.keys()]
del counts[0]
del hex_colors[0]

print(counts)
plt.figure(figsize = (8, 6))
plt.pie(counts.values(), labels = hex_colors, colors = hex_colors)

### Image Display Function

In [ ]:
def display(comp,name,ogName = "Original Image isup",img = img_test):
    f = plt.figure(figsize=(15,15))
    f.add_subplot(2, 1, 1).set_title(f'{ogName} = {isup}')
    plt.imshow(img, cmap = "gray")
    f.add_subplot(2, 1, 2).set_title(f'{name} isup = {isup}')
    plt.imshow(comp, cmap = "gray")
    return plt.show()

### Image Sharpening
<hr>

In [ ]:
import cv2
img = cv2.blur(img_test, (3, 3))

kernel = [
            [-1, -1, -1],
            [-1, 9, -1],
            [-1, -1, -1]
]

kernel = np.array(kernel)

sharpened = cv2.filter2D(img, -1, kernel)

display(sharpened, "sharpened")

## Convolution kernel

In [ ]:

kernel = [
            [0, 1, 0],
            [1, -4, 1],
            [0, 1, 0]
]

kernel = np.array(kernel)

filtered = cv2.filter2D(sharpened, -1, kernel)

In [ ]:
f = plt.figure(figsize=(15,15))
f.add_subplot(2, 1, 1).set_title(f'Original Image isup = {isup}')
plt.imshow(img_test, cmap = "gray")
f.add_subplot(2, 1, 2).set_title(f'Filtered Image isup = {isup}')
plt.imshow(filtered, cmap = "gray")
plt.show()

#### Will use this filtered image for further analysis

### Image Segmentation

In [ ]:
# Segmentation
gray = cv2.cvtColor(filtered, cv2.COLOR_RGB2GRAY)
ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

In [ ]:
f = plt.figure(figsize=(15,15))
f.add_subplot(2, 1, 1).set_title(f'Original Image isup = {isup}')
plt.imshow(img_test, cmap = "gray")
f.add_subplot(2, 1, 2).set_title(f'PreSegment Image isup = {isup}')
plt.imshow(thresh, cmap = "gray")
plt.show()

In [ ]:
# Further noise removal
kernel = np.ones((3, 3), np.uint8)
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

# sure background area
sure_bg = cv2.dilate(opening, kernel, iterations=3)

# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
ret, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg, sure_fg)

#Displaying segmented back ground
display(sure_bg, 'Segmented Background')

In [ ]:
#will not use segmeted code from above but insteaf idetify unique clusters in the image

img_test = img_array[0]

In [ ]:
# Marker labelling
ret, markers = cv2.connectedComponents(thresh)

# Add one to all labels so that sure background is not 0, but 1
markers = markers + 1

# Now, mark the region of unknown with zero
markers[unknown == 255] = 0

markers = cv2.watershed(img_test, markers)
img_test[markers == -1] = [255, 0, 0]

# Displaying markers on the image
display(markers,  'Marked')

### Testing Pipeline for new Image (ISUP 4)
<hr>

In [ ]:
isup4 = img_array[3]

In [ ]:
plt.imshow(isup4)

In [ ]:
isup = train_df.iloc[3]['isup_grade']

In [ ]:
modified_image = cv2.resize(isup4, (600, 400), interpolation = cv2.INTER_AREA)
modified_image = modified_image.reshape(modified_image.shape[0]*modified_image.shape[1], 3)

#color seperation
clf = KMeans(n_clusters =8)
labels = clf.fit_predict(modified_image)

counts = Counter(labels)

center_colors = clf.cluster_centers_
# We get ordered colors by iterating through the keys
ordered_colors = [center_colors[i] for i in counts.keys()]
hex_colors = [RGB2HEX(ordered_colors[i]) for i in counts.keys()]
rgb_colors = [ordered_colors[i] for i in counts.keys()]

del counts[0]
del hex_colors[0]

print(counts)
plt.figure(figsize = (8, 6))
plt.pie(counts.values(), labels = hex_colors, colors = hex_colors)

hex_colors

In [ ]:
counts

In [ ]:
counts.values()

In [ ]:
testing_image = isup4

In [ ]:
def image_sharpening(image,ogImage):
    img = cv2.blur(image, (3, 3))
    kernel = [
            [-1, -1, -1],
            [-1, 9, -1],
            [-1, -1, -1]
    ]

    kernel = np.array(kernel)
    image = cv2.filter2D(img, -1, kernel)

    return display(image, 'sharpened', f"Original Image isup = {isup}",ogImage ) 

def convolution_kernel(image,ogImage):
    kernel = [
            [0, 1, 0],
            [1, -4, 1],
            [0, 1, 0]
    ]

    kernel = np.array(kernel)
    image = cv2.filter2D(image, -1, kernel)


    return display(image, 'Convolution Kernel',  f"Original Image isup = {isup}",ogImage) 
    
def segmentation(image,ogImage, num):
    
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    # Further noise removal
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)


    # sure background area
    sure_bg = cv2.dilate(opening, kernel, iterations=3)

    # Finding sure foreground area
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    ret, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)
    
    image = img_array[num]
    # Marker labelling
    ret, markers = cv2.connectedComponents(thresh)

    # Add one to all labels so that sure background is not 0, but 1
    markers = markers + 1

    # Now, mark the region of unknown with zero
    markers[unknown == 255] = 0

    markers = cv2.watershed(img_test, markers)
    image[markers == -1] = [255, 0, 0]

    # Displaying markers on the image
    return display(markers,  'Marked',  f"Original Image isup = {isup}",ogImage)

In [ ]:
image_sharpening(testing_image,isup4)

In [ ]:
convolution_kernel(testing_image,isup4)

In [ ]:
segmentation(testing_image,isup4,3)

### Testing for ISUP 1

In [ ]:
isup1 = img_array[9]

In [ ]:
isup = train_df.iloc[9]['isup_grade']
plt.imshow(isup1)

In [ ]:
modified_image = cv2.resize(isup1, (600, 400), interpolation = cv2.INTER_AREA)
modified_image = modified_image.reshape(modified_image.shape[0]*modified_image.shape[1], 3)

#color seperation
clf = KMeans(n_clusters =8)
labels = clf.fit_predict(modified_image)

counts = Counter(labels)

center_colors = clf.cluster_centers_
# We get ordered colors by iterating through the keys
ordered_colors = [center_colors[i] for i in counts.keys()]
hex_colors = [RGB2HEX(ordered_colors[i]) for i in counts.keys()]
rgb_colors = [ordered_colors[i] for i in counts.keys()]
del counts[0]
del hex_colors[0]

print(counts)

plt.figure(figsize = (8, 6))
plt.pie(counts.values(), labels = hex_colors, colors = hex_colors)

hex_colors

In [ ]:
counts

In [ ]:
testing_image = isup1

In [ ]:
image_sharpening(testing_image,isup1)

In [ ]:
convolution_kernel(testing_image,isup1)

In [ ]:
segmentation(testing_image,isup1,9)